In [6]:
import pandas as pd
base = pd.read_csv('credit_data.csv')
base.head()

,i#clientid,income,age,loan,c#default
0,1,66155.925095,59.017015,8106.532131,0
1,2,34415.153966,48.117153,6564.745018,0
2,3,57317.170063,63.108049,8020.953296,0
3,4,42709.534201,45.751972,6103.642260,0
4,5,66952.688845,18.584336,8770.099235,1


In [7]:
base.shape

(2000, 5)

In [8]:
base = base.drop('i#clientid', axis = 1)
base.head()

,income,age,loan,c#default
0,66155.925095,59.017015,8106.532131,0
1,34415.153966,48.117153,6564.745018,0
2,57317.170063,63.108049,8020.953296,0
3,42709.534201,45.751972,6103.642260,0
4,66952.688845,18.584336,8770.099235,1


In [9]:
base = base.dropna() ## apaga registros com valores nao preechidos
base.shape

(1997, 4)

In [10]:
# Padronização dos valores

from sklearn.preprocessing import StandardScaler
scaler_x = StandardScaler()
base[['income', 'age', 'loan']] = scaler_x.fit_transform(base[['income', 'age', 'loan']])
base.head()

,income,age,loan,c#default
0,1.453898,1.336861,1.201907,0
1,-0.762398,0.536639,0.695744,0
2,0.836733,1.637207,1.173812,0
3,-0.183244,0.362998,0.544366,0
4,1.509532,-1.631534,1.419754,1


In [11]:
X = base.drop('c#default', axis = 1)
y = base['c#default']

In [12]:
X.head()

,income,age,loan
0,1.453898,1.336861,1.201907
1,-0.762398,0.536639,0.695744
2,0.836733,1.637207,1.173812
3,-0.183244,0.362998,0.544366
4,1.509532,-1.631534,1.419754


In [13]:
y.head()

0    0
1    0
2    0
3    0
4    1
Name: c#default, dtype: int64

In [14]:
X.columns

Index(['income', 'age', 'loan'], dtype='object')

In [15]:
import tensorflow as tf

# Percorrendo as colunas e colocando nome em cada uma delas
colunas = [tf.feature_column.numeric_column(key = column) for column in X.columns]

C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Douglas\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [16]:
print(colunas[2])

NumericColumn(key='loan', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


In [17]:
from sklearn.model_selection import train_test_split
X_treinamento, X_teste, y_treinamento, y_teste = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [18]:
X_treinamento.shape

(1397, 3)

In [19]:
X_teste.shape

(600, 3)

In [20]:
funcao_treinamento = tf.estimator.inputs.pandas_input_fn(x = X_treinamento,
                                                        y = y_treinamento,
                                                        batch_size = 8, 
                                                        num_epochs = None,
                                                        shuffle = True)

In [21]:
# Criando a rede neural DNN
# Duas camadas ocultas, cada uma com 4 neuronios
classificador = tf.estimator.DNNClassifier(feature_columns = colunas, hidden_units = [4, 4])
classificador.train(input_fn = funcao_treinamento, steps = 1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\Douglas\\AppData\\Local\\Temp\\tmp37z88a4v', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000275022E4608>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
Instructions for updating:
Use Variable.read_value. Var

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\Douglas\AppData\Local\Temp\tmp37z88a4v\model.ckpt.
INFO:tensorflow:loss = 5.614525, step = 1
INFO:tensorflow:global_step/sec: 473.898
INFO:tensorflow:loss = 6.616407, step = 101 (0.214 sec)
INFO:tensorflow:global_step/sec: 729.873
INFO:tensorflow:loss = 0.9967163, step = 201 (0.135 sec)
INFO:tensorflow:global_step/sec: 751.759
INFO:tensorflow:loss = 0.9694543, step = 301 (0.134 sec)
INFO:tensorflow:global_step/sec: 775.207
INFO:tensorflow:loss = 4.0381823, step = 401 (0.128 sec)
INFO:tensorflow:global_step/sec: 729.873
INFO:tensorflow:

In [22]:
funcao_teste = tf.estimator.inputs.pandas_input_fn(x = X_teste, y = y_teste,
                                              batch_size = 8, num_epochs = 1000,
                                              shuffle = False)

In [23]:
metricas_teste = classificador.evaluate(input_fn = funcao_teste, steps = 1000)

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-17T15:30:17Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from C:\Users\Douglas\AppData\Local\Temp\tmp37z88a4v\model.ckpt-1000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Evaluation [100/1000]
INFO:tensorflow:Evaluation [200/1000]
INFO:tensorflow:Evaluation [300/1000]
INFO:tensorflow:Evaluation [400/1000]
INFO:tensorflow:Evaluation [500/1000]
INFO:tensorflow:Evaluation [600/1000]
INFO:tensorflow:Evaluation [700/1000]
INFO:tensorflow:Evaluation [800/1000]
INFO:tensorflow:Evaluation [900/1000]
INFO:tensorflow:Evaluation [1000/1000]
INFO:tensorflow:Finished evaluation at 2020-08-17-15:30:20
INFO:tensorflow:Saving dict for glob

In [24]:
metricas_teste

{'accuracy': 0.981375,
 'accuracy_baseline': 0.8635,
 'auc': 0.9990175,
 'auc_precision_recall': 0.9940422,
 'average_loss': 0.08453502,
 'label/mean': 0.1365,
 'loss': 0.67628014,
 'precision': 0.9865841,
 'prediction/mean': 0.09717051,
 'recall': 0.8754579,
 'global_step': 1000}